# Data wrangling 

In [1]:
import preprocessing as pre
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn import metrics
import plotly.express as px
from sklearn import preprocessing

First let's import the neuro dataset (neuropsychological assessment)

In [2]:
neuro = pd.read_excel("Neuropsy.xlsx", na_values='.').dropna()
neuro['ID']='id_'+neuro['ID'].astype(str)
neuro.rename(columns={'ID':'id'}, inplace=True)# rename ID to id (to match eeg dataset)
neuro['subtype'] = neuro.apply(lambda x: pre.categorize_subtypes(x['inat'], x['hyper']), axis=1)# create a column with categorized subtybes (based on clinical thresholds)
neuro['is_inat'] = neuro['subtype'].apply(lambda x: 0 if x=='hyper' else 1) # create a columns to label prensence of inat and hyper
neuro['is_hyper'] = neuro['subtype'].apply(lambda x: 0 if x=='inat' else 1)
neuro

id  Age  Gender   cIM   cHR   cIE   cSC  inat  hyper  Aqtot  Aqaudi  \
0     id_1   21       1  17.0  31.0  29.0   9.0  18.0   20.0   90.0    91.0   
1     id_3   20       1  10.0   5.0  13.0   1.0   8.0    5.0   27.0    34.0   
2     id_4   18       1  26.0  17.0   7.0  15.0  23.0   11.0   93.0    89.0   
3     id_7   23       1  24.0   8.0   6.0  14.0  19.0    3.0   86.0    66.0   
5    id_11   21       1  16.0  26.0  13.0  10.0  17.0   13.0   45.0    33.0   
..     ...  ...     ...   ...   ...   ...   ...   ...    ...    ...     ...   
95  id_602   19       1  25.0  14.0  20.0  18.0  21.0    8.0   80.0    87.0   
96  id_603   19       1  32.0  15.0  14.0  15.0  22.0    9.0  111.0   103.0   
97  id_700   18       2  13.0  33.0  14.0  11.0  18.0   15.0   83.0    80.0   
98  id_701   19       1  22.0  17.0  15.0  12.0  15.0   11.0    0.0    20.0   
99  id_702   18       1  17.0  14.0  10.0   9.0  12.0    3.0   91.0    88.0   

    Aqvis  RCQtot  RCQaudi  RCQvis subtype  is_inat  is_hyper  
0    92.0    94.0     80.0   110.0   mixed        1         1  
1    41.0    25.0     31.0    38.0   mixed        1         1  
2    96.0    90.0     92.0    90.0    inat        1         0  
3   112.0    94.0     90.0   100.0    inat        1         0  
5    78.0    69.0     51.0    94.0   mixed        1         1  
..    ...     ...      ...     ...     ...      ...       ...  
95   77.0    71.0     71.0    79.0    inat        1         0  
96  118.0    63.0     78.0    56.0    inat        1         0  
97   89.0    70.0     72.0    75.0   mixed        1         1  
98    0.0    29.0     32.0    42.0   mixed        1         1  
99   95.0    94.0     87.0   104.0    inat        1         0  

[96 rows x 18 columns]

In [3]:
neuro.Gender.value_counts() #verify sample gender 

1    57
2    39
Name: Gender, dtype: int64

Import all all excel files into one df following function from our .py file

In [4]:
eeg = pre.process_all_excel_files()
eeg['fft_abs_power'] = eeg['fft_abs_power'].astype(float)
eeg['id'] = eeg['id'].astype(int)
eeg['id'] ='id_'+eeg['id'].astype(str)
print(eeg.shape)
eeg.head(5)

(7676, 5)


electrode brain_oscillation  fft_abs_power  freq_band      id
0       FP1             Delta      15.565495  1.0-4.0Hz  id_134
1       FP2             Delta      13.945462  1.0-4.0Hz  id_134
2        F7             Delta      10.874886  1.0-4.0Hz  id_134
3        F3             Delta      13.581803  1.0-4.0Hz  id_134
4        Fz             Delta      15.644595  1.0-4.0Hz  id_134

## Setting up df for analysis

Drop hyper subtype as under represented n=2 on a n=96 sample

In [5]:
neuro.drop( neuro[ neuro['subtype'] == 'hyper' ].index , inplace=True)
neuro.subtype.value_counts()

inat     48
mixed    46
Name: subtype, dtype: int64

Merge dataframes togeter and keep columns of interest

In [6]:
#inner join to keep ids present in both dataframes
df = pd.merge(eeg, neuro[['id', 'Gender','subtype','is_inat','is_hyper', 'inat','hyper']], on='id', how='inner')
print(df.shape)
len(df.id.unique())
df.head()

(7144, 11)


electrode brain_oscillation  fft_abs_power  freq_band      id  Gender  \
0       FP1             Delta      15.565495  1.0-4.0Hz  id_134       1   
1       FP2             Delta      13.945462  1.0-4.0Hz  id_134       1   
2        F7             Delta      10.874886  1.0-4.0Hz  id_134       1   
3        F3             Delta      13.581803  1.0-4.0Hz  id_134       1   
4        Fz             Delta      15.644595  1.0-4.0Hz  id_134       1   

  subtype  is_inat  is_hyper  inat  hyper  
0    inat        1         0  10.0    3.0  
1    inat        1         0  10.0    3.0  
2    inat        1         0  10.0    3.0  
3    inat        1         0  10.0    3.0  
4    inat        1         0  10.0    3.0

Verify value counts for the variables

In [7]:
print(eeg.brain_oscillation.value_counts())
print(eeg.freq_band.value_counts())
print(eeg.info())

Beta     1919
Theta    1919
Delta    1919
Alpha    1919
Name: brain_oscillation, dtype: int64
12.0-25.0Hz    1919
8.0-12.0Hz     1919
1.0-4.0Hz      1919
4.0-8.0Hz      1919
Name: freq_band, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7676 entries, 0 to 7675
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   electrode          7676 non-null   object 
 1   brain_oscillation  7676 non-null   object 
 2   fft_abs_power      7676 non-null   float64
 3   freq_band          7676 non-null   object 
 4   id                 7676 non-null   object 
dtypes: float64(1), object(4)
memory usage: 300.0+ KB
None


# Some data vizualisation 

In [8]:
graph_data = df.groupby(['id', 'brain_oscillation', 'subtype']).agg({'fft_abs_power':'mean'}).reset_index()
graph_data.sort_values(['brain_oscillation','fft_abs_power'] ,inplace=True)
fig = px.bar(graph_data, x='id', y='fft_abs_power', facet_col='brain_oscillation', color='subtype')
fig.show()

# PCA

## PCA on  each electrode 

### Retain one electrode at a time

#### PCA targetting prediction of gender by features of electrode, brain oscillation, and scores at inat and hyper

Categorical variables now need to be encoded 

In [9]:
le = preprocessing.LabelEncoder()
#encode features with scitkit learn encoder
#https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html
df_encoded = df
df_encoded = df_encoded[['electrode', 'brain_oscillation', 'fft_abs_power', 'subtype']].copy()
df_encoded['subtype'] = le.fit_transform(df_encoded['subtype'])
df_encoded['electrode'].unique()

array(['FP1', 'FP2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'T3', 'C3', 'Cz', 'C4',
       'T4', 'T5', 'P3', 'Pz', 'P4', 'T6', 'O1', 'O2'], dtype=object)

Extract one electrode at a time to run PCA

In [24]:
# Retain only the Fp1 channel
eeg_Fp1 = df_encoded.loc[df_encoded['electrode'] == 'FP1'] 
# select only columns of interest 
eeg_Fp1 = eeg_Fp1[['brain_oscillation', 'fft_abs_power', 'subtype']]
# pivot table
eeg_Fp1_wide = pd.pivot_table(eeg_Fp1, index= 'subtype', columns = ['brain_oscillation'], values='fft_abs_power')
eeg_Fp1_wide

brain_oscillation     Alpha      Beta      Delta    Theta
fft_abs_power      6.267651  8.862711  17.628118  7.24943

In [13]:
eeg_Fp1.shape

(376, 3)

In [27]:
#label
subtype = eeg_Fp1['subtype']
subtype = subtype.to_numpy()
subtype = np.reshape(subtype,(376,1))
subtype =pd.DataFrame(subtype)
subtype

0
0    0
1    0
2    0
3    0
4    1
..  ..
371  0
372  0
373  0
374  0
375  0

[376 rows x 1 columns]

Separate features from the target 'subtype'

In [ ]:
subtype_labels = eeg_Fp1['subtype']# extract column
subtype_labels =subtype_labels.to_numpy()# convert to numpy
subtype_labels = np.reshape(subtype_labels,(1,4))# reshape so it can be merged to the df
subtype_labels =pd.DataFrame(subtype_labels) # reconvert as pd df

Merge labels and features columns together

In [ ]:
eeg_dataset= pd.concat([features,gender_labels], axis=1).rename(columns={0: "labels"})
eeg_dataset

In [ ]:
##############

In [ ]:
hyper_labels = df['is_hyper']
inat_labels = df['is_inat']
gender_labels = df['Gender']

In [ ]:
gender_labels =gender_labels.to_numpy()

In [ ]:
import numpy as np
labels = np.reshape(gender_labels,(7144,1))

In [ ]:
gender_labels =pd.DataFrame(gender_labels)
gender_labels

In [ ]:
eeg_dataset= pd.concat([features,gender_labels], axis=1).rename(columns={0: "labels"})
eeg_dataset

In [ ]:
#features  #= eeg_dataset.loc[eeg_dataset['electrode'] == 7]
#eeg_dataset['electrode'] = eeg_dataset.apply(lambda x: pre.pca_electrode, axis=1)

#### Standardize the data

In [ ]:
from sklearn.preprocessing import StandardScaler
# Separating out the features
x = eeg_dataset.loc[:, features].values
                    
# Separating out the target
y = eeg_dataset.loc[:,["labels"]].values
                    
# Standardizing the features
x = StandardScaler().fit_transform(x)

#### PCA Projection to 2D

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)

principalComponents = pca.fit_transform(x)

principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

In [ ]:
finalDf = pd.concat([principalDf, eeg_dataset[['labels']]], axis = 1)
finalDf

In [ ]:
finalDf=finalDf.replace({'labels': 1}, {'labels': 'female'}, regex=True)
finalDf=finalDf.replace({'labels': 2}, {'labels': 'male'}, regex=True)
finalDf

#### Visualize 2D Projection

In [ ]:
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)

labels = ['female', 'male']
colors = ['r', 'g']
for target, color in zip(labels,colors):
    indicesToKeep = finalDf['labels'] == target
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50)
ax.legend(labels)
ax.grid()

#### Explained variance

In [ ]:
pca.explained_variance_ratio_

# KNN

## Predicting gender

In [ ]:
le = preprocessing.LabelEncoder()
#encode features with scitkit learn encoder
#https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html
features = df
features = features[['electrode', 'brain_oscillation', 'fft_abs_power', 'inat', 'hyper']].copy()
# encoding series with categorical variables
features['electrode'] = le.fit_transform(features['electrode'])
features['brain_oscillation'] = le.fit_transform(features['brain_oscillation'])

In [ ]:
#create train, test sets
X_train, X_test, y_train, y_test = train_test_split(features, gender_labels, test_size=0.2, random_state=4)

#Create KNN Classifier
knn = KNeighborsClassifier(n_neighbors=2)

#Train the model using the training sets
knn.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = knn.predict(X_test)

In [ ]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
plt.figure()
sns.pairplot(dataset.drop("Id", axis=1), hue = "Species", size=3, markers=["o", "s", "D"])
plt.show()

### Permutation 

In [ ]:
import numpy as np
from sklearn.model_selection import permutation_test_score

score, permutation_scores, pvalue = permutation_test_score(
    knn, X_train, y_train, scoring="accuracy",  n_permutations=100, n_jobs=1)

print("Classification score %s (pvalue : %s)" % (score, pvalue))

In [ ]:
import matplotlib.pyplot as plt
plt.hist(permutation_scores, 20, label='Permutation scores',
         edgecolor='black')
ylim = plt.ylim()
plt.plot(2 * [score], ylim, '--g', linewidth=3,
         label='Classification Score'
         ' (pvalue %s)' % pvalue)



#### Confusion matrix

In [ ]:
metrics.confusion_matrix(y_test, y_pred, normalize='true')
#https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html?highlight=confusion%20matrix

In [ ]:
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
titles_options = [("Confusion matrix, without normalization", None),
                  ("Normalized confusion matrix", 'true')]
for title, normalize in titles_options:
    disp = metrics.plot_confusion_matrix(knn, X_test, y_test,
                                 display_labels=['1','2'],
                                 cmap=plt.cm.Blues,
                                 normalize=normalize)
    disp.ax_.set_title(title)

    print(title)
    print(disp.confusion_matrix)

plt.show()

### 

## Predicting hyper

In [ ]:
#create train, test sets
X_train, X_test, y_train, y_test = train_test_split(features, hyper_labels, test_size=0.2, random_state=4)

#Create KNN Classifier
knn = KNeighborsClassifier(n_neighbors=2)

#Train the model using the training sets
knn.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = knn.predict(X_test)

In [ ]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
import numpy as np
from sklearn.model_selection import permutation_test_score

score, permutation_scores, pvalue = permutation_test_score(
    knn, X_train, y_train, scoring="accuracy",  n_permutations=100, n_jobs=1)

print("Classification score %s (pvalue : %s)" % (score, pvalue))

In [ ]:
import matplotlib.pyplot as plt
plt.hist(permutation_scores, 20, label='Permutation scores',
         edgecolor='black')
ylim = plt.ylim()
plt.plot(2 * [score], ylim, '--g', linewidth=3,
         label='Classification Score'
         ' (pvalue %s)' % pvalue)


## Predicting inat

### Encoding of features 

1.2 pas adequat comme echantillon compose a 98 % de inat, donc non valide

In [ ]:
le = preprocessing.LabelEncoder()
#encode features with scitkit learn encoder
#https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html
features = df
features = features[['electrode', 'brain_oscillation', 'fft_abs_power']].copy()
# encoding series with categorical variables
features['electrode'] = le.fit_transform(features['electrode'])
features['brain_oscillation'] = le.fit_transform(features['brain_oscillation'])


In [ ]:
#create train, test sets
X_train, X_test, y_train, y_test = train_test_split(features, inat_labels, test_size=0.2, random_state=4)

#Create KNN Classifier
knn = KNeighborsClassifier(n_neighbors=2)

#Train the model using the training sets
knn.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = knn.predict(X_test)

In [ ]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
import numpy as np
from sklearn.model_selection import permutation_test_score

score, permutation_scores, pvalue = permutation_test_score(
    knn, X_train, y_train, scoring="accuracy",  n_permutations=100, n_jobs=1)

print("Classification score %s (pvalue : %s)" % (score, pvalue))

In [ ]:
import matplotlib.pyplot as plt
plt.hist(permutation_scores, 20, label='Permutation scores',
         edgecolor='black')
ylim = plt.ylim()
plt.plot(2 * [score], ylim, '--g', linewidth=3,
         label='Classification Score'
         ' (pvalue %s)' % pvalue)
